## hp_activation.py

This code is from the file `hp_activation.py`.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar  8 23:58:43 2021

@author: lihen
"""

#import relevant packages and set some important parameters
import time
start_time = time.time()
import random
import tensorflow as tf
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
from tensorboard.plugins.hparams import api as hp
# Python 3.5
import numpy as np
import pandas as pd
import sys
from tensorflow.keras.callbacks import ReduceLROnPlateau
np.set_printoptions(threshold=sys.maxsize)
pd.set_option("display.max_rows", None, "display.max_columns", None)
result = pd.read_pickle('./naca4_clcd_turb_st_3para.pkl', compression=None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#set the random seeds required for initialization
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1615400000)
   tf.random.set_seed(1615400000)
   np.random.seed(1615400000)
   random.seed(1615400000)
gg=1000


#load data from naca4_clcd_turb_st_3para.pkl
inp_reno=[]
inp_aoa=[]
inp_para=[]


out_cm=[]
out_cd=[]
out_cl=[]


out_cm.extend(result[0])   
out_cd.extend(result[1])
out_cl.extend(result[2])

inp_reno.extend(result[3])
inp_aoa.extend(result[4])
inp_para.extend(result[5])

out_cm=np.asarray(out_cm)/0.188171
out_cd=np.asarray(out_cd)/0.2466741
out_cl=np.asarray(out_cl)/1.44906


out_cd=np.asarray(out_cd)
out_cl=np.asarray(out_cl)

inp_reno=np.asarray(inp_reno)
inp_aoa=np.asarray(inp_aoa)
inp_para=np.asarray(inp_para)/np.array([6,6,30])

N= len(out_cm)
print(N)
I = np.arange(N)
np.random.shuffle(I)
n=N

#normalize the numeral values such that the max value is 1
inp_reno=inp_reno/100000.
inp_aoa=inp_aoa/14.0

my_inp=np.concatenate((inp_reno[:,None],inp_aoa[:,None],inp_para[:,:]),axis=1)
my_out=np.concatenate((out_cd[:,None],out_cl[:,None]),axis=1)
## Training sets
xtr0 = my_inp[I][:n]
ttr1 = my_out[I][:n]

#list of activation functions to train model
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['elu',
                                                     'exponential',
                                                    
                                                     'hard_sigmoid',
                                                     'relu',
                                                     'selu',
                                                     'sigmoid',
                                                     'softmax',
                                                     'softplus',
                                                     'softsign',
                                                     'swish',
                                                     'tanh']))
METRIC_MSE = 'Mean Squared Error'

with tf.summary.create_file_writer('logs/activation').as_default():
    hp.hparams_config(
    hparams=[HP_ACTIVATION],
    metrics=[hp.Metric(METRIC_MSE, display_name='val_loss')],
    )
    
epochs = list(range(0,gg))

#this function sets the activation function of the model for training based on the list given.
def train_test_model(hparams):
    activation_name = hparams[HP_ACTIVATION]
    if activation_name == "elu":
        a = tf.keras.activations.elu
    elif activation_name == "exponential":
        a = tf.keras.activations.exponential
    elif activation_name == "gelu":
        a = tf.keras.activations.gelu
    elif activation_name == "hard_sigmoid":
        a = tf.keras.activations.hard_sigmoid
    elif activation_name == "relu":
        a = tf.keras.activations.relu
    elif activation_name == "selu":
        a = tf.keras.activations.selu
    elif activation_name == "sigmoid":
        a = tf.keras.activations.sigmoid
    elif activation_name == "softmax":
        a = tf.keras.activations.softmax
    elif activation_name == "softplus":
        a = tf.keras.activations.softplus 
    elif activation_name == "softsign":
        a = tf.keras.activations.softsign
    elif activation_name == "swish":
        a = tf.keras.activations.swish   
    elif activation_name == "tanh":
        a = tf.keras.activations.tanh
    else:
        raise ValueError("unexpected optimizer name: %r" % (activation_name))
    xx = 90
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(5,)))
    model.add(tf.keras.layers.Dense(xx, kernel_initializer='random_normal', activation=a))
    model.add(tf.keras.layers.Dense(xx, activation=a))
    model.add(tf.keras.layers.Dense(xx, activation=a))
    model.add(tf.keras.layers.Dense(xx, activation=a))
    model.add(tf.keras.layers.Dense(xx, activation=a))
    model.add(tf.keras.layers.Dense(2, activation=None))
    model.summary()
    
    
    model.compile(
        optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError()]
        )
  # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=run_name)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, mode='min',verbose=1 ,patience=100, min_lr=1.0e-8)
    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001), loss='mean_squared_error')
    history = model.fit([xtr0], [ttr1], validation_split=0.1,callbacks=[reduce_lr], epochs=gg)
    mse = np.array(history.history['val_loss'])
    return mse

#records the loss function of this model 
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    mse1 = train_test_model(hparams)
    for idx, epoch in enumerate(epochs):
        mse = mse1[idx]
        tf.summary.scalar(METRIC_MSE, mse, step=epoch+1)
    
session_num = 0
#tensorboard --logdir='C:/Users/lihen/projects/tf-gpu-MNIST/logs/hparam_tuning5layer' is the path to called Tensorboard for comparing models

#print details and keep logs upon script execution
for activation in HP_ACTIVATION.domain.values:
    hparams = {
        HP_ACTIVATION: activation
        }
    run_name = "{}".format(activation)
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/activation/' + run_name, hparams)
    session_num += 1
    reset_random_seeds()
 




















## hp_batch.py

This code is from the file `hp_batch.py`.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar  9 00:04:43 2021

@author: lihen
"""

#import relevant packages and set some important parameters
import time
start_time = time.time()
import random
import tensorflow as tf
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
from tensorboard.plugins.hparams import api as hp
# Python 3.5
import numpy as np
import pandas as pd
import sys
from tensorflow.keras.callbacks import ReduceLROnPlateau
np.set_printoptions(threshold=sys.maxsize)
pd.set_option("display.max_rows", None, "display.max_columns", None)
result = pd.read_pickle('./naca4_clcd_turb_st_3para.pkl', compression=None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1615400000)
   tf.random.set_seed(1615400000)
   np.random.seed(1615400000)
   random.seed(1615400000)
gg=1000
#load data from naca4_clcd_turb_st_3para.pkl
inp_reno=[]
inp_aoa=[]
inp_para=[]


out_cm=[]
out_cd=[]
out_cl=[]


out_cm.extend(result[0])   
out_cd.extend(result[1])
out_cl.extend(result[2])

inp_reno.extend(result[3])
inp_aoa.extend(result[4])
inp_para.extend(result[5])

out_cm=np.asarray(out_cm)/0.188171
out_cd=np.asarray(out_cd)/0.2466741
out_cl=np.asarray(out_cl)/1.44906


out_cd=np.asarray(out_cd)
out_cl=np.asarray(out_cl)

inp_reno=np.asarray(inp_reno)
inp_aoa=np.asarray(inp_aoa)
inp_para=np.asarray(inp_para)/np.array([6,6,30])

N= len(out_cm)
print(N)
I = np.arange(N)
np.random.shuffle(I)
n=N

#normalize the numeral values such that the max value is 1
inp_reno=inp_reno/100000.
inp_aoa=inp_aoa/14.0

my_inp=np.concatenate((inp_reno[:,None],inp_aoa[:,None],inp_para[:,:]),axis=1)
my_out=np.concatenate((out_cd[:,None],out_cl[:,None]),axis=1)
## Training sets
xtr0 = my_inp[I][:n]
ttr1 = my_out[I][:n]


#list of discrete batch sizes to train model
HP_BATCH= hp.HParam('batch_size', hp.Discrete([16,32,64,128,256,512]))
METRIC_MSE = 'Mean Squared Error'

with tf.summary.create_file_writer('logs/batch').as_default():
    hp.hparams_config(
    hparams=[HP_BATCH],
    metrics=[hp.Metric(METRIC_MSE, display_name='val_loss')],
  )

epochs = list(range(0,gg))

#this function sets the batch size of the model for training based on the list given.
def train_test_model(hparams):
    xx = 90
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(5,)))
    model.add(tf.keras.layers.Dense(xx, kernel_initializer='random_normal', activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(2, activation=None))
    model.summary()
    
    size = hparams[HP_BATCH] 
    if size == 16:
        b = 16
    elif size == 32:
        b = 32
    elif size == 64:
        b= 64
    elif size == 128:
        b= 128
    elif size == 256:
        b= 256
    elif size == 512:
        b= 512
    else:
        raise ValueError("unexpected size: %r" % (size))
    
    model.compile(
        optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError()]
        )
  # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=run_name)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, mode='min',verbose=1 ,patience=100, min_lr=1.0e-8)
    history = model.fit([xtr0], [ttr1], validation_split=0.1,callbacks=[reduce_lr], batch_size=b, epochs=gg)
    mse = np.array(history.history['val_loss'])
    return mse

#records the loss function of this model 
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    mse1 = train_test_model(hparams)
    for idx, epoch in enumerate(epochs):
        mse = mse1[idx]
        tf.summary.scalar(METRIC_MSE, mse, step=epoch+1)
    
session_num = 0
#tensorboard --logdir='C:/Users/lihen/projects/tf-gpu-MNIST/logs/hparam_tuning5layer' is the path to called Tensorboard for comparing models

#print details and keep logs upon script execution
for batch in HP_BATCH.domain.values:
    hparams = {
        HP_BATCH: batch
        }
    run_name = "%d" % (batch)
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/batch/' + run_name, hparams)
    session_num += 1
    reset_random_seeds()
     




















## hp_learningrate.py

This code is from the file `hp_learningrate.py`.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  2 14:19:39 2021

@author: lihen
"""

#import relevant packages and set some important parameters
import time
start_time = time.time()
import random
import tensorflow as tf
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
from tensorboard.plugins.hparams import api as hp
# Python 3.5
import numpy as np
import pandas as pd
import sys
from tensorflow.keras.callbacks import ReduceLROnPlateau
np.set_printoptions(threshold=sys.maxsize)
pd.set_option("display.max_rows", None, "display.max_columns", None)
result = pd.read_pickle('./naca4_clcd_turb_st_3para.pkl', compression=None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#load data from naca4_clcd_turb_st_3para.pkl
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1615400000)
   tf.random.set_seed(1615400000)
   np.random.seed(1615400000)
   random.seed(1615400000)
gg=1000
#load data
inp_reno=[]
inp_aoa=[]
inp_para=[]


out_cm=[]
out_cd=[]
out_cl=[]


out_cm.extend(result[0])   
out_cd.extend(result[1])
out_cl.extend(result[2])

inp_reno.extend(result[3])
inp_aoa.extend(result[4])
inp_para.extend(result[5])

out_cm=np.asarray(out_cm)/0.188171
out_cd=np.asarray(out_cd)/0.2466741
out_cl=np.asarray(out_cl)/1.44906


out_cd=np.asarray(out_cd)
out_cl=np.asarray(out_cl)

inp_reno=np.asarray(inp_reno)
inp_aoa=np.asarray(inp_aoa)
inp_para=np.asarray(inp_para)/np.array([6,6,30])

N= len(out_cm)
print(N)
I = np.arange(N)
np.random.shuffle(I)
n=N

#normalize the numeral values such that the max value is 1
inp_reno=inp_reno/100000.
inp_aoa=inp_aoa/14.0

my_inp=np.concatenate((inp_reno[:,None],inp_aoa[:,None],inp_para[:,:]),axis=1)
my_out=np.concatenate((out_cd[:,None],out_cl[:,None]),axis=1)
## Training sets
xtr0 = my_inp[I][:n]
ttr1 = my_out[I][:n]

#list of learning rates to train model
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['nadam']))
HP_L_RATE= hp.HParam('learning_rate', hp.Discrete([0.00001, 
                                                   0.000025, 
                                                   0.00005, 
                                                   0.000075, 
                                                   0.0001,
                                                   0.00025,
                                                   0.0005,
                                                   0.00075,
                                                   0.001,
                                                   0.0025,
                                                   0.005,
                                                   0.0075,
                                                   0.01]))
METRIC_MSE = 'Mean Squared Error'

with tf.summary.create_file_writer('logs/learning rate').as_default():
    hp.hparams_config(
    hparams=[HP_L_RATE],
    metrics=[hp.Metric(METRIC_MSE, display_name='val_loss')],
  )

epochs = list(range(0,gg))

#this function sets the learning rate of the model for training based on the list given. The base activation function is relu
def train_test_model(hparams):
    xx = 90
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(5,)))
    model.add(tf.keras.layers.Dense(xx, kernel_initializer='random_normal', activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(2, activation=None))
    model.summary()
    optimizer_name = hparams[HP_OPTIMIZER]
    lr = hparams[HP_L_RATE]
    if optimizer_name == "nadam":
        o = tf.keras.optimizers.Nadam(learning_rate=lr)
    else:
        raise ValueError("unexpected optimizer name: %r" % (optimizer_name,))
    
    model.compile(
        optimizer=o,
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError()]
        )
  # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=run_name)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, mode='min',verbose=1 ,patience=100, min_lr=1.0e-8)
    history = model.fit([xtr0], [ttr1], validation_split=0.1,callbacks=[reduce_lr], epochs=gg)
    mse = np.array(history.history['val_loss'])
    return mse

#records the loss function of this model 
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    mse1 = train_test_model(hparams)
    for idx, epoch in enumerate(epochs):
        mse = mse1[idx]
        tf.summary.scalar(METRIC_MSE, mse, step=epoch+1)
    

#tensorboard --logdir='C:/Users/lihen/projects/tf-gpu-MNIST/logs/hparam_tuning5layer'
for optimizer in HP_OPTIMIZER.domain.values:
    for learning_rate in HP_L_RATE.domain.values:
        hparams = {
            HP_OPTIMIZER: optimizer,
            HP_L_RATE: learning_rate
            }
        run_name = "{}".format(learning_rate)
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        reset_random_seeds()
        run('logs/learning rate/' + run_name, hparams)
        
    




















## hp_neurons.py

This code is from the file `hp_neurons.py`.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar  9 00:01:05 2021

@author: lihen
"""

#import relevant packages and set some important parameters
import time
start_time = time.time()
import numpy as np
import random
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
# Python 3.5
import os
import pandas as pd
import sys
from tensorflow.keras.callbacks import ReduceLROnPlateau
np.set_printoptions(threshold=sys.maxsize)
pd.set_option("display.max_rows", None, "display.max_columns", None)
result = pd.read_pickle(r'C:/Users/lihen/projects/tf-gpu-MNIST/naca4_clcd_turb_st_3para.pkl', compression=None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
layers = [1,2,3,4,5,6,7,8,9,10]

#set the random seeds required for initialization
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1615400000)
   tf.random.set_seed(1615400000)
   np.random.seed(1615400000)
   random.seed(1615400000)
   os.environ['TF_DETERMINISTIC_OPS'] = '1'
gg=1000

#load data from naca4_clcd_turb_st_3para.pkl
inp_reno=[]
inp_aoa=[]
inp_para=[]


out_cm=[]
out_cd=[]
out_cl=[]


out_cm.extend(result[0])   
out_cd.extend(result[1])
out_cl.extend(result[2])

inp_reno.extend(result[3])
inp_aoa.extend(result[4])
inp_para.extend(result[5])

out_cm=np.asarray(out_cm)/0.188171
out_cd=np.asarray(out_cd)/0.2466741
out_cl=np.asarray(out_cl)/1.44906


out_cd=np.asarray(out_cd)
out_cl=np.asarray(out_cl)

inp_reno=np.asarray(inp_reno)
inp_aoa=np.asarray(inp_aoa)
inp_para=np.asarray(inp_para)/np.array([6,6,30])

N= len(out_cm)
print(N)
I = np.arange(N)
np.random.shuffle(I)
n=N

#normalize the numeral values such that the max value is 1
inp_reno=inp_reno/100000.
inp_aoa=inp_aoa/14.0

my_inp=np.concatenate((inp_reno[:,None],inp_aoa[:,None],inp_para[:,:]),axis=1)
my_out=np.concatenate((out_cd[:,None],out_cl[:,None]),axis=1)
## Training sets
xtr0 = my_inp[I][:n]
ttr1 = my_out[I][:n]

#Hyperparameters set from 1 to 10 layers with neurons in each layer in multiples of 10, up to 100
HP_NEURONS = hp.HParam('neurons', hp.Discrete(range(10,101,10)))
HP_LAYERS = hp.HParam('layer', hp.Discrete([1,2,3,4,5,6,7,8,9,10]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_L_RATE= hp.HParam('learning_rate', hp.Discrete([2.5e-4]))
METRIC_MSE = 'Mean Squared Error'

with tf.summary.create_file_writer('logs/1 hidden layer').as_default():
    hp.hparams_config(
    hparams=[HP_LAYERS, HP_NEURONS, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_MSE, display_name='val_loss')],
  )

epochs = list(range(0,gg))

#this function builds the layers and neurons of the model for training based on the list given.
def train_test_model(hparams, layer,rundir):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(5,)))
    model.add(tf.keras.layers.Dense(hparams[HP_NEURONS], kernel_initializer='random_normal', activation=tf.keras.activations.relu))
    if layer > 1: 
        for i in range(layer-1):
             model.add(tf.keras.layers.Dense(hparams[HP_NEURONS], activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(2, activation=None))
    model.summary()
    optimizer_name = hparams[HP_OPTIMIZER]
    learning_rate = hparams[HP_L_RATE]
    if optimizer_name == "adam":
        o = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "sgd":
        o = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        raise ValueError("unexpected optimizer name: %r" % (optimizer_name,))
    model.compile(
        optimizer=o,
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError()]
        )
    # logdir=rundir
    # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, profile_batch = 100000000)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, mode='min',verbose=0 ,patience=100, min_lr=1.0e-8)
    history = model.fit([xtr0], [ttr1], validation_split=0.1,callbacks=[reduce_lr], epochs=gg,shuffle=False)
    mse = np.array(history.history['val_loss'])
    return mse

#records the loss function of this model 
def run(run_dir, hparams, layer):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        mse1 = train_test_model(hparams, layer, run_dir)
        for idx, epoch in enumerate(epochs):
            mse = mse1[idx]
            tf.summary.scalar(METRIC_MSE, mse, step=epoch+1)
    
#tensorboard --logdir='C:/Users/lihen/projects/tf-gpu-MNIST/logs/hparam_tuning5layer'
for layer in HP_LAYERS.domain.values:
    for neurons in HP_NEURONS.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
            for learning_rate in HP_L_RATE.domain.values:
                hparams = {
                    HP_LAYERS: layer,
                    HP_NEURONS: neurons,
                    HP_OPTIMIZER: optimizer,
                    HP_L_RATE: learning_rate
                    }
                run_name = "{}".format(neurons) + " Neurons"
                print('--- Starting trial: %s' % run_name)
                print("{} layer ".format(layer))
                print({h.name: hparams[h] for h in hparams})
                reset_random_seeds()
                run('logs/{} hidden layer '.format(layer) + run_name, hparams, layer)







## hp_optimizer.py

This code is from the file `hp_optimizer.py`.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  2 14:19:39 2021

@author: lihen
"""

#import relevant packages and set some important parameters
import time
start_time = time.time()
import random
import tensorflow as tf
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
from tensorboard.plugins.hparams import api as hp
# Python 3.5
import numpy as np
import pandas as pd
import sys
from tensorflow.keras.callbacks import ReduceLROnPlateau
np.set_printoptions(threshold=sys.maxsize)
pd.set_option("display.max_rows", None, "display.max_columns", None)
result = pd.read_pickle('./naca4_clcd_turb_st_3para.pkl', compression=None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#set the random seeds required for initialization
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1615400000)
   tf.random.set_seed(1615400000)
   np.random.seed(1615400000)
   random.seed(1615400000)
gg=1000

#load data from naca4_clcd_turb_st_3para.pkl
inp_reno=[]
inp_aoa=[]
inp_para=[]


out_cm=[]
out_cd=[]
out_cl=[]


out_cm.extend(result[0])   
out_cd.extend(result[1])
out_cl.extend(result[2])

inp_reno.extend(result[3])
inp_aoa.extend(result[4])
inp_para.extend(result[5])

out_cm=np.asarray(out_cm)/0.188171
out_cd=np.asarray(out_cd)/0.2466741
out_cl=np.asarray(out_cl)/1.44906


out_cd=np.asarray(out_cd)
out_cl=np.asarray(out_cl)

inp_reno=np.asarray(inp_reno)
inp_aoa=np.asarray(inp_aoa)
inp_para=np.asarray(inp_para)/np.array([6,6,30])

N= len(out_cm)
print(N)
I = np.arange(N)
np.random.shuffle(I)
n=N

#normalize
inp_reno=inp_reno/100000.
inp_aoa=inp_aoa/14.0

my_inp=np.concatenate((inp_reno[:,None],inp_aoa[:,None],inp_para[:,:]),axis=1)
my_out=np.concatenate((out_cd[:,None],out_cl[:,None]),axis=1)
## Training sets
xtr0 = my_inp[I][:n]
ttr1 = my_out[I][:n]

#list of optimizers to train model
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['ftrl',
    'adam', 
    'adadelta',
    'adagrad', 
    'adamax',
    'sgd',
    'nadam',
    'rmsprop'
    ]))
# HP_L_RATE= hp.HParam('learning_rate', hp.Discrete([2.5e-4]))
METRIC_MSE = 'Mean Squared Error'

with tf.summary.create_file_writer('logs/optimizer/').as_default():
  hp.hparams_config(
    hparams=[HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_MSE, display_name='val_loss')],
  )

epochs = list(range(0,gg))

#this function sets the optimizer of the model for training based on the list given.
def train_test_model(hparams):
    xx = 90
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(5,)))
    model.add(tf.keras.layers.Dense(xx, kernel_initializer='random_normal', activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(2, activation=None))
    model.summary()
    optimizer_name = hparams[HP_OPTIMIZER]
    if optimizer_name == "adam":
        o = tf.keras.optimizers.Adam()
    elif optimizer_name == "adadelta":
        o = tf.keras.optimizers.Adadelta()
    elif optimizer_name == "adagrad":
        o = tf.keras.optimizers.Adagrad()
    elif optimizer_name == "adamax":
        o = tf.keras.optimizers.Adamax()
    elif optimizer_name == "sgd":
        o = tf.keras.optimizers.SGD()
    elif optimizer_name == "nadam":
        o = tf.keras.optimizers.Nadam()
    elif optimizer_name == "rmsprop":
        o = tf.keras.optimizers.RMSprop()
    elif optimizer_name == "ftrl":
        o = tf.keras.optimizers.Ftrl()
    else:
        raise ValueError("unexpected optimizer name: %r" % (optimizer_name))
    
    model.compile(
        optimizer=o,
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError()]
        )
  # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=run_name)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, mode='min',verbose=1 ,patience=100, min_lr=1.0e-8)
    history = model.fit([xtr0], [ttr1], validation_split=0.1,callbacks=[reduce_lr], epochs=gg, shuffle=False)
    mse = np.array(history.history['val_loss'])
    return mse

#records the loss function of this model 
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    mse1 = train_test_model(hparams)
    for idx, epoch in enumerate(epochs):
        mse = mse1[idx]
        tf.summary.scalar(METRIC_MSE, mse, step=epoch+1)
    
#tensorboard --logdir='C:/Users/lihen/projects/tf-gpu-MNIST/logs/hparam_tuning5layer'

#print details and keep logs upon script execution
for optimizer in HP_OPTIMIZER.domain.values:
    hparams = {
        HP_OPTIMIZER: optimizer
        }
    run_name = "{}".format(optimizer)
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    reset_random_seeds()
    run('logs/optimizer/' + run_name, hparams)







































## main.py

This code is from the file `main.py`.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb  3 23:18:38 2021

@author: lihen
"""

#import relevant packages and set some important parameters
import time
start_time = time.time()
import numpy as np
import random
import tensorflow as tf
# Python 3.5
import os
import pandas as pd
import sys
import pickle
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
np.set_printoptions(threshold=sys.maxsize)
pd.set_option("display.max_rows", None, "display.max_columns", None)
result = pd.read_pickle(r'./naca4_clcd_turb_st_3para.pkl', compression=None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#set the random seeds required for initialization

os.environ['PYTHONHASHSEED']=str(1615400000)
tf.random.set_seed(1615400000)
np.random.seed(1615400000)
random.seed(1615400000)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
    
#load data from naca4_clcd_turb_st_3para.pkl    
#load data
inp_reno=[]
inp_aoa=[]
inp_para=[]

out_cm=[]
out_cd=[]
out_cl=[]

out_cm.extend(result[0])   
out_cd.extend(result[1])
out_cl.extend(result[2])

inp_reno.extend(result[3])
inp_aoa.extend(result[4])
inp_para.extend(result[5])

out_cm=np.asarray(out_cm)/0.188171
out_cd=np.asarray(out_cd)/0.2466741
out_cl=np.asarray(out_cl)/1.44906


out_cd=np.asarray(out_cd)
out_cl=np.asarray(out_cl)

inp_reno=np.asarray(inp_reno)
inp_aoa=np.asarray(inp_aoa)
inp_para=np.asarray(inp_para)/np.array([6,6,30])


# ---------ML PART:-----------#
#shuffle data
N= len(out_cm)
print(N)
I = np.arange(N)
np.random.shuffle(I)
n=N

#normalize the numeral values such that the max value is 1
inp_reno=inp_reno/100000.
inp_aoa=inp_aoa/14.0

my_inp=np.concatenate((inp_reno[:,None],inp_aoa[:,None],inp_para[:,:]),axis=1)
my_out=np.concatenate((out_cd[:,None],out_cl[:,None]),axis=1)

## Training sets
xtr0= my_inp[I][:n]
ttr1 = my_out[I][:n]

# Multilayer Perceptron
# create model
xx= 90
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(5,)))
model.add(tf.keras.layers.Dense(xx, kernel_initializer='random_normal', activation=tf.keras.activations.swish))
model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.swish))
model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.swish))
model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.swish))
model.add(tf.keras.layers.Dense(xx, activation=tf.keras.activations.swish))
model.add(tf.keras.layers.Dense(2, activation=None))

#callbacks
# reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, mode='min',verbose=1 ,patience=100, min_lr=1.0e-8)

#prevents overtraining when the ANN has converged at a local minimum
e_stop = EarlyStopping(monitor='loss', min_delta=1.0e-8, patience=200, verbose=0, mode='auto')

filepath="./model_optimized_notreduced/model_sf_{epoch:02d}_{loss:.8f}_{val_loss:.8f}.hdf5"

chkpt= ModelCheckpoint(filepath, monitor='val_loss', verbose=0,\
                                save_best_only=False, save_weights_only=False, mode='auto', period=100)

# Compile model
opt = tf.keras.optimizers.Nadam(learning_rate=0.001)

model.compile(optimizer= opt, loss=tf.keras.losses.MeanSquaredError())

hist = model.fit([xtr0], [ttr1], validation_split=0.1, batch_size=32, callbacks=[e_stop,chkpt],verbose=1,shuffle=False, epochs=10000)

#save model
model.save('./model_optimized_notreduced/final_sf.hdf5') 

print("\n") 
print("loss = %f to %f"%(np.asarray(hist.history["loss"][:1]),np.asarray(hist.history["loss"][-1:])))
print("\n")
print("val_loss = %f to %f"%(np.asarray(hist.history["val_loss"][:1]),np.asarray(hist.history["val_loss"][-1:])))
print("\n")
print("--- %s seconds ---" % (time.time() - start_time))

data1=[hist.history]
with open('./model_optimized_notreduced/hist.pkl', 'wb') as outfile:
    pickle.dump(data1, outfile, pickle.HIGHEST_PROTOCOL)
model.summary()
#%%
model.summary()













